In [1]:
import math
import torch
from torch import nn
from torch import Tensor
from torch.nn  import functional as F
import gpytorch
from matplotlib import pyplot as plt
import sys
from decimal import Decimal
sys.path.append("..")
import vvkernels as vvk
import vvk_rbfkernel as vvk_rbf
import vvmeans as vvm
import vvlikelihood as vvll
from vfield import VField
import numpy as np
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
def check_dist(y1, y2,tol):
    index = range(y2.shape[0])
       
    index_del = []
    check = False
    for ii in range (y1.shape[0]):
#         if (x[ii,0] < 0 or x[ii,1] <0):
#             check = True
#             index_del.append(ii)
        for jj in range(y2.shape[0]):
            if (torch.norm(y1[ii] - y2[jj])) <= tol:
                check = True
                index_del.append(jj)
    if check == True :
        index_del = (np.unique(index_del,))
        index_del = np.array((index_del), dtype = int)
                
        index_ = np.delete(index, index_del)
        index_ = np.array(index_)
        y2_new = torch.zeros(index_.shape[0], y2.shape[1])
        jj = 0
        for ii in index_:
                    
            y2_new[jj] = y2[ii]
            jj = jj + 1
    else:
        index_ = index
        y2_new = y2
    return y2_new, check
    

In [3]:
vf = VField()
f_target = vf.tgt_vec
sample_size = 100
D = vf.D
N = vf.N

def g_theta(sample_size, D):
    loc = np.random.random_sample((sample_size,D))
    return Tensor(loc)
train_x = g_theta(sample_size, D)
train_y = torch.zeros([sample_size, N])




for i in range(sample_size):
    train_y[i] = Tensor(vf(train_x[i]))+ torch.randn(Tensor(vf(train_x[i])).size()) * 0.005
    

In [4]:
def vfield_(x):
    x = x.reshape(x.shape[0],D)
    out = torch.zeros(x.shape[0], N)
    for i in range(x.shape[0]):
        out[i] = Tensor(vf(x[i])) + torch.randn(Tensor(vf(x[i])).size()) * 0.005
    return out

In [5]:
def stopping_criteria(tol_vector, f_target, lower_bound, upper_bound):
    lower_tol_vector = f_target - tol_vector
    upper_tol_vector = f_target + tol_vector
    SUCCESS = True
    FAILURE = False
    for i in range(f_target.shape[0]):
            if (lower_bound[i] < lower_tol_vector[i]) or  (upper_bound[i] > upper_tol_vector[i]):
                SUCCESS = False  
            if ((lower_bound[i] > upper_tol_vector[i]) or  (upper_bound[i] < lower_tol_vector[i])):
                FAILURE = True
    return SUCCESS, FAILURE

In [6]:

x_train = train_x #loc #torch.linspace(0, 1, 10)
y_train = train_y #v  #torch.stack([torch.sin(train_x * (2 * math.pi)) + torch.randn(train_x.size()) * 0.2,torch.cos(train_x * (2 * math.pi)) + torch.randn(train_x.size()) * 0.2,], -1)

class MultitaskGPModel(gpytorch.models.ExactGP):
    def __init__(self, train_x, train_y, likelihood):
        super(MultitaskGPModel, self).__init__(train_x, train_y, likelihood)
        a = torch.ones(2,2)
        chol_q = torch.tril(a)
        self.mean_module = vvm.TensorProductSubMean(gpytorch.means.ConstantMean(), num_tasks = 2)
#         self.covar_module = gpytorch.kernels.MultitaskKernel(
#             gpytorch.kernels.RBFKernel(), num_tasks=2, rank=1
#         )
        self.covar_module = vvk.TensorProductKernel(vvk_rbf.vvkRBFKernel(), a[0,0], a[1,0], a[1,1], num_tasks = 2, rank =1,  task_covar_prior=None)

    def forward(self, x):
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x,x)
        return gpytorch.distributions.MultitaskMultivariateNormal(mean_x, covar_x)
    
    


In [7]:
###hyperparameters optimization###
def hyper_opti(g_theta1, agg_data, training_iter):
    likelihood = vvll.TensorProductLikelihood(num_tasks = 2)

    model = MultitaskGPModel(g_theta1, agg_data, likelihood)
    model.train()
    likelihood.train()

    optimizer = torch.optim.Adam(model.parameters(),  lr=0.1)  # Includes GaussianLikelihood parameters
    mll = gpytorch.mlls.ExactMarginalLogLikelihood(likelihood, model)
    for i in range(training_iter):
        optimizer.zero_grad()
        output = model(g_theta1)
        #output_ll = likelihood(output)

        #loss = -likelihood.get_mll(agg_data,output_ll)
        loss = -mll(output, agg_data)
        loss.backward()

        print('Iter %d/%d - Loss hyperparam: %.3f' % (i + 1, training_iter, loss.item()))
        optimizer.step()

    return model, likelihood


In [8]:
class param_opti(nn.Module):
    def __init__(self, sample):
        super(param_opti, self).__init__()
        #loc = np.random.random_sample((loc_size,2))
        self.g_theta2 = nn.Parameter(Tensor(sample))
        
    def forward(self):
        
        return (self.g_theta2)

In [9]:
def conduct_param_opti(x,loc_sample, f_target,g_theta1, agg_data, model, likelihood, training_iter):
    model.eval()
    likelihood.eval()
    _par = param_opti(loc_sample)
    
    optimizer = torch.optim.Adam(_par.parameters(), lr=0.007)
    
    for j in range(training_iter):
        optimizer.zero_grad()
        g_theta2 = _par.forward()

        loss1, lower_bound, upper_bound = likelihood.get_ell(agg_data,f_target,x, g_theta1, g_theta2, model, likelihood)
        loss1 = -1 * loss1
        loss1.backward(retain_graph=True)
        print('Iter %d/%d - Loss theta2: %.3f' % (j + 1, training_iter, loss1.item()))
        optimizer.step()
          
    return loss1, g_theta2, lower_bound, upper_bound

In [10]:
class design_opti(nn.Module):
    def __init__(self, _x):
        super(design_opti, self).__init__()
#         a = _x[0,0]
#         b = _x[0,1]
#         self.x_design_1 = nn.Parameter(a)
#         self.x_design_2 = nn.Parameter(b)
        self.x_design = nn.Parameter(_x)
    def forward(self):
#         x_design = torch.zeros(1,2)
#         x_design[0,0] = self.x_design_1
#         x_design[0,1] = self.x_design_2
        return (self.x_design)

In [11]:
def conduct_design_opti(x0,loc_sample, f_target, g_theta1, agg_data, model, likelihood, training_design_iter, training_param_iter):
    design = design_opti(x0)
    loc_sample0 = loc_sample
    x_d = design.forward()
    def closure():
        optimizer.zero_grad()
        loss2, g_theta2_out,lower_bound, upper_bound = conduct_param_opti(x_d,loc_sample, f_target,g_theta1, agg_data, model, likelihood, training_param_iter)
        loss2.backward(retain_graph=True)
        
        return loss2
        
        
        
    optimizer = torch.optim.LBFGS(design.parameters(), lr=1., history_size=100, max_iter=10, line_search_fn="strong_wolfe")
    optimizer.step(closure)

    x_d = design.forward()
    loss2, g_theta2_out,lower_bound, upper_bound = conduct_param_opti(x_d,loc_sample, f_target,g_theta1, agg_data, model, likelihood, training_param_iter)
    print('Loss design: %.3f' % ( loss2))
    print(optimizer.state)
    return x_d, g_theta2_out, lower_bound, upper_bound
    
    

In [12]:
iter_hp = 50
iter_design = 20
iter_param = 50

f_target = Tensor(vf.tgt_vec)
f_target = f_target.reshape(f_target.shape[0],1)
tol_vector = 0.005 * torch.ones(f_target.shape)


loc_size = 50
loc_sample = np.random.random_sample((loc_size,2))



g_theta1 = x_train

agg_data = y_train.flatten()

# agg_data.append(y_train.flatten())
# agg_data = torch.cat(agg_data)

x0 = Tensor(np.array([0.5,0.7]))
x0 = x0.reshape(1,2)
x00 = x0
vec_x = x00
SUCCESS = False
FAILURE = False
iter = 0
tol  = 0.009
while(SUCCESS == False and iter < 50):
    
    
    print('START HYPERPARAMETERS optimization')
    model, likelihood = hyper_opti(g_theta1,agg_data,iter_hp)
    
    print('END HYPERPARAMETERS optimization')
    print(iter)
    x0_new,g_theta2,lower_bound, upper_bound = conduct_design_opti(x0, loc_sample, f_target, g_theta1, agg_data, model, likelihood, iter_design,iter_param)
    print(lower_bound)
    print(upper_bound)
    print(f_target-tol_vector)
    print(f_target+tol_vector)
    loc_sample = np.random.random_sample((loc_size,2))
    x0 = x0_new #Tensor(np.random.random_sample((1,2))) #x0_new
    vec_x = torch.cat([vec_x, x0_new])
    print(x0_new)
#     model.eval()
#     likelihood.eval()
#     pred = likelihood(model(g_theta2))
    #1y_train_new = (pred.mean)
    #agg_data.append(y_train_new.flatten())
    g_theta2_detach = g_theta2.detach()
    new_data = vfield_(g_theta2_detach)
    agg_data = torch.cat([agg_data, new_data.flatten()], 0)
    g_theta1= torch.cat([g_theta1, g_theta2], 0)
    #loc_sample, check = check_dist(g_theta1, Tensor(loc_sample), tol)
    SUCCESS, FAILURE = stopping_criteria(tol_vector, f_target, lower_bound, upper_bound)
    iter = iter + 1

print(x0_new)
    

START HYPERPARAMETERS optimization
Iter 1/50 - Loss hyperparam: 0.852
Iter 2/50 - Loss hyperparam: 0.811
Iter 3/50 - Loss hyperparam: 0.770
Iter 4/50 - Loss hyperparam: 0.730
Iter 5/50 - Loss hyperparam: 0.689
Iter 6/50 - Loss hyperparam: 0.648
Iter 7/50 - Loss hyperparam: 0.607
Iter 8/50 - Loss hyperparam: 0.565
Iter 9/50 - Loss hyperparam: 0.523
Iter 10/50 - Loss hyperparam: 0.481
Iter 11/50 - Loss hyperparam: 0.439
Iter 12/50 - Loss hyperparam: 0.395
Iter 13/50 - Loss hyperparam: 0.352
Iter 14/50 - Loss hyperparam: 0.308
Iter 15/50 - Loss hyperparam: 0.263
Iter 16/50 - Loss hyperparam: 0.218
Iter 17/50 - Loss hyperparam: 0.173
Iter 18/50 - Loss hyperparam: 0.127
Iter 19/50 - Loss hyperparam: 0.081
Iter 20/50 - Loss hyperparam: 0.034
Iter 21/50 - Loss hyperparam: -0.013
Iter 22/50 - Loss hyperparam: -0.061
Iter 23/50 - Loss hyperparam: -0.108
Iter 24/50 - Loss hyperparam: -0.156
Iter 25/50 - Loss hyperparam: -0.205
Iter 26/50 - Loss hyperparam: -0.253
Iter 27/50 - Loss hyperparam: -0

Iter 41/50 - Loss theta2: -3.529
Iter 42/50 - Loss theta2: -3.532
Iter 43/50 - Loss theta2: -3.535
Iter 44/50 - Loss theta2: -3.537
Iter 45/50 - Loss theta2: -3.539
Iter 46/50 - Loss theta2: -3.541
Iter 47/50 - Loss theta2: -3.543
Iter 48/50 - Loss theta2: -3.546
Iter 49/50 - Loss theta2: -3.549
Iter 50/50 - Loss theta2: -3.551
Iter 1/50 - Loss theta2: -2.458
Iter 2/50 - Loss theta2: -2.558
Iter 3/50 - Loss theta2: -2.652
Iter 4/50 - Loss theta2: -2.736
Iter 5/50 - Loss theta2: -2.809
Iter 6/50 - Loss theta2: -2.881
Iter 7/50 - Loss theta2: -2.943
Iter 8/50 - Loss theta2: -2.997
Iter 9/50 - Loss theta2: -3.048
Iter 10/50 - Loss theta2: -3.096
Iter 11/50 - Loss theta2: -3.138
Iter 12/50 - Loss theta2: -3.177
Iter 13/50 - Loss theta2: -3.205
Iter 14/50 - Loss theta2: -3.236
Iter 15/50 - Loss theta2: -3.266
Iter 16/50 - Loss theta2: -3.289
Iter 17/50 - Loss theta2: -3.310
Iter 18/50 - Loss theta2: -3.329
Iter 19/50 - Loss theta2: -3.347
Iter 20/50 - Loss theta2: -3.364
Iter 21/50 - Loss t

Iter 48/50 - Loss theta2: -3.545
Iter 49/50 - Loss theta2: -3.548
Iter 50/50 - Loss theta2: -3.550
Iter 1/50 - Loss theta2: -2.472
Iter 2/50 - Loss theta2: -2.570
Iter 3/50 - Loss theta2: -2.660
Iter 4/50 - Loss theta2: -2.743
Iter 5/50 - Loss theta2: -2.819
Iter 6/50 - Loss theta2: -2.887
Iter 7/50 - Loss theta2: -2.950
Iter 8/50 - Loss theta2: -3.007
Iter 9/50 - Loss theta2: -3.053
Iter 10/50 - Loss theta2: -3.106
Iter 11/50 - Loss theta2: -3.143
Iter 12/50 - Loss theta2: -3.179
Iter 13/50 - Loss theta2: -3.212
Iter 14/50 - Loss theta2: -3.244
Iter 15/50 - Loss theta2: -3.269
Iter 16/50 - Loss theta2: -3.295
Iter 17/50 - Loss theta2: -3.315
Iter 18/50 - Loss theta2: -3.336
Iter 19/50 - Loss theta2: -3.352
Iter 20/50 - Loss theta2: -3.369
Iter 21/50 - Loss theta2: -3.383
Iter 22/50 - Loss theta2: -3.396
Iter 23/50 - Loss theta2: -3.409
Iter 24/50 - Loss theta2: -3.422
Iter 25/50 - Loss theta2: -3.432
Iter 26/50 - Loss theta2: -3.438
Iter 27/50 - Loss theta2: -3.450
Iter 28/50 - Loss t

Iter 8/50 - Loss theta2: -3.030
Iter 9/50 - Loss theta2: -3.083
Iter 10/50 - Loss theta2: -3.129
Iter 11/50 - Loss theta2: -3.168
Iter 12/50 - Loss theta2: -3.206
Iter 13/50 - Loss theta2: -3.236
Iter 14/50 - Loss theta2: -3.266
Iter 15/50 - Loss theta2: -3.295
Iter 16/50 - Loss theta2: -3.317
Iter 17/50 - Loss theta2: -3.338
Iter 18/50 - Loss theta2: -3.358
Iter 19/50 - Loss theta2: -3.373
Iter 20/50 - Loss theta2: -3.391
Iter 21/50 - Loss theta2: -3.404
Iter 22/50 - Loss theta2: -3.418
Iter 23/50 - Loss theta2: -3.429
Iter 24/50 - Loss theta2: -3.441
Iter 25/50 - Loss theta2: -3.449
Iter 26/50 - Loss theta2: -3.460
Iter 27/50 - Loss theta2: -3.469
Iter 28/50 - Loss theta2: -3.477
Iter 29/50 - Loss theta2: -3.483
Iter 30/50 - Loss theta2: -3.491
Iter 31/50 - Loss theta2: -3.495
Iter 32/50 - Loss theta2: -3.501
Iter 33/50 - Loss theta2: -3.506
Iter 34/50 - Loss theta2: -3.510
Iter 35/50 - Loss theta2: -3.515
Iter 36/50 - Loss theta2: -3.519
Iter 37/50 - Loss theta2: -3.522
Iter 38/50 -

Iter 17/50 - Loss theta2: -4.309
Iter 18/50 - Loss theta2: -4.309
Iter 19/50 - Loss theta2: -4.309
Iter 20/50 - Loss theta2: -4.310
Iter 21/50 - Loss theta2: -4.310
Iter 22/50 - Loss theta2: -4.310
Iter 23/50 - Loss theta2: -4.311
Iter 24/50 - Loss theta2: -4.310
Iter 25/50 - Loss theta2: -4.311
Iter 26/50 - Loss theta2: -4.311
Iter 27/50 - Loss theta2: -4.311
Iter 28/50 - Loss theta2: -4.312
Iter 29/50 - Loss theta2: -4.312
Iter 30/50 - Loss theta2: -4.312
Iter 31/50 - Loss theta2: -4.312
Iter 32/50 - Loss theta2: -4.313
Iter 33/50 - Loss theta2: -4.313
Iter 34/50 - Loss theta2: -4.313
Iter 35/50 - Loss theta2: -4.313
Iter 36/50 - Loss theta2: -4.313
Iter 37/50 - Loss theta2: -4.313
Iter 38/50 - Loss theta2: -4.314
Iter 39/50 - Loss theta2: -4.313
Iter 40/50 - Loss theta2: -4.314
Iter 41/50 - Loss theta2: -4.314
Iter 42/50 - Loss theta2: -4.314
Iter 43/50 - Loss theta2: -4.314
Iter 44/50 - Loss theta2: -4.314
Iter 45/50 - Loss theta2: -4.314
Iter 46/50 - Loss theta2: -4.315
Iter 47/50

...

Iter 36/50 - Loss theta2: -11.015
Iter 37/50 - Loss theta2: -11.016
Iter 38/50 - Loss theta2: -11.016
Iter 39/50 - Loss theta2: -11.017
Iter 40/50 - Loss theta2: -11.018
Iter 41/50 - Loss theta2: -11.019
Iter 42/50 - Loss theta2: -11.019
Iter 43/50 - Loss theta2: -11.019
Iter 44/50 - Loss theta2: -11.020
Iter 45/50 - Loss theta2: -11.021
Iter 46/50 - Loss theta2: -11.022
Iter 47/50 - Loss theta2: -11.022
Iter 48/50 - Loss theta2: -11.023
Iter 49/50 - Loss theta2: -11.023
Iter 50/50 - Loss theta2: -11.024
Iter 1/50 - Loss theta2: -11.015
Iter 2/50 - Loss theta2: -11.017
Iter 3/50 - Loss theta2: -11.017
Iter 4/50 - Loss theta2: -11.017
Iter 5/50 - Loss theta2: -11.018
Iter 6/50 - Loss theta2: -11.018
Iter 7/50 - Loss theta2: -11.019
Iter 8/50 - Loss theta2: -11.020
Iter 9/50 - Loss theta2: -11.020
Iter 10/50 - Loss theta2: -11.021
Iter 11/50 - Loss theta2: -11.022
Iter 12/50 - Loss theta2: -11.023
Iter 13/50 - Loss theta2: -11.023
Iter 14/50 - Loss theta2: -11.023
Iter 15/50 - Loss theta

Iter 1/50 - Loss hyperparam: 0.773
Iter 2/50 - Loss hyperparam: 0.735
Iter 3/50 - Loss hyperparam: 0.695
Iter 4/50 - Loss hyperparam: 0.656
Iter 5/50 - Loss hyperparam: 0.616
Iter 6/50 - Loss hyperparam: 0.575
Iter 7/50 - Loss hyperparam: 0.534
Iter 8/50 - Loss hyperparam: 0.493
Iter 9/50 - Loss hyperparam: 0.451
Iter 10/50 - Loss hyperparam: 0.408
Iter 11/50 - Loss hyperparam: 0.365
Iter 12/50 - Loss hyperparam: 0.321
Iter 13/50 - Loss hyperparam: 0.276
Iter 14/50 - Loss hyperparam: 0.231
Iter 15/50 - Loss hyperparam: 0.186
Iter 16/50 - Loss hyperparam: 0.140
Iter 17/50 - Loss hyperparam: 0.093
Iter 18/50 - Loss hyperparam: 0.046
Iter 19/50 - Loss hyperparam: -0.001
Iter 20/50 - Loss hyperparam: -0.049
Iter 21/50 - Loss hyperparam: -0.097
Iter 22/50 - Loss hyperparam: -0.146
Iter 23/50 - Loss hyperparam: -0.195
Iter 24/50 - Loss hyperparam: -0.244
Iter 25/50 - Loss hyperparam: -0.294
Iter 26/50 - Loss hyperparam: -0.344
Iter 27/50 - Loss hyperparam: -0.394
Iter 28/50 - Loss hyperparam

Iter 45/50 - Loss theta2: -4.487
Iter 46/50 - Loss theta2: -4.489
Iter 47/50 - Loss theta2: -4.488
Iter 48/50 - Loss theta2: -4.489
Iter 49/50 - Loss theta2: -4.489
Iter 50/50 - Loss theta2: -4.489
Iter 1/50 - Loss theta2: -7.967
Iter 2/50 - Loss theta2: -7.961
Iter 3/50 - Loss theta2: -7.968
Iter 4/50 - Loss theta2: -7.969
Iter 5/50 - Loss theta2: -7.969
Iter 6/50 - Loss theta2: -7.978
Iter 7/50 - Loss theta2: -7.977
Iter 8/50 - Loss theta2: -7.983
Iter 9/50 - Loss theta2: -7.984
Iter 10/50 - Loss theta2: -7.983
Iter 11/50 - Loss theta2: -7.989
Iter 12/50 - Loss theta2: -7.991
Iter 13/50 - Loss theta2: -7.990
Iter 14/50 - Loss theta2: -7.993
Iter 15/50 - Loss theta2: -7.993
Iter 16/50 - Loss theta2: -7.994
Iter 17/50 - Loss theta2: -7.997
Iter 18/50 - Loss theta2: -8.000
Iter 19/50 - Loss theta2: -7.998
Iter 20/50 - Loss theta2: -8.000
Iter 21/50 - Loss theta2: -8.004
Iter 22/50 - Loss theta2: -8.002
Iter 23/50 - Loss theta2: -8.003
Iter 24/50 - Loss theta2: -8.002
Iter 25/50 - Loss t

Iter 40/50 - Loss theta2: -11.119
Iter 41/50 - Loss theta2: -11.119
Iter 42/50 - Loss theta2: -11.119
Iter 43/50 - Loss theta2: -11.121
Iter 44/50 - Loss theta2: -11.120
Iter 45/50 - Loss theta2: -11.121
Iter 46/50 - Loss theta2: -11.121
Iter 47/50 - Loss theta2: -11.121
Iter 48/50 - Loss theta2: -11.124
Iter 49/50 - Loss theta2: -11.123
Iter 50/50 - Loss theta2: -11.125
Iter 1/50 - Loss theta2: -11.184
Iter 2/50 - Loss theta2: -11.184
Iter 3/50 - Loss theta2: -11.184
Iter 4/50 - Loss theta2: -11.185
Iter 5/50 - Loss theta2: -11.187
Iter 6/50 - Loss theta2: -11.188
Iter 7/50 - Loss theta2: -11.188
Iter 8/50 - Loss theta2: -11.189
Iter 9/50 - Loss theta2: -11.190
Iter 10/50 - Loss theta2: -11.190
Iter 11/50 - Loss theta2: -11.192
Iter 12/50 - Loss theta2: -11.191
Iter 13/50 - Loss theta2: -11.192
Iter 14/50 - Loss theta2: -11.193
Iter 15/50 - Loss theta2: -11.194
Iter 16/50 - Loss theta2: -11.194
Iter 17/50 - Loss theta2: -11.196
Iter 18/50 - Loss theta2: -11.197
Iter 19/50 - Loss theta

Iter 33/50 - Loss theta2: -11.270
Iter 34/50 - Loss theta2: -11.271
Iter 35/50 - Loss theta2: -11.273
Iter 36/50 - Loss theta2: -11.274
Iter 37/50 - Loss theta2: -11.275
Iter 38/50 - Loss theta2: -11.275
Iter 39/50 - Loss theta2: -11.277
Iter 40/50 - Loss theta2: -11.278
Iter 41/50 - Loss theta2: -11.278
Iter 42/50 - Loss theta2: -11.278
Iter 43/50 - Loss theta2: -11.281
Iter 44/50 - Loss theta2: -11.283
Iter 45/50 - Loss theta2: -11.283
Iter 46/50 - Loss theta2: -11.283
Iter 47/50 - Loss theta2: -11.284
Iter 48/50 - Loss theta2: -11.286
Iter 49/50 - Loss theta2: -11.290
Iter 50/50 - Loss theta2: -11.285
Iter 1/50 - Loss theta2: -11.228
Iter 2/50 - Loss theta2: -11.230
Iter 3/50 - Loss theta2: -11.230
Iter 4/50 - Loss theta2: -11.230
Iter 5/50 - Loss theta2: -11.232
Iter 6/50 - Loss theta2: -11.234
Iter 7/50 - Loss theta2: -11.231
Iter 8/50 - Loss theta2: -11.236
Iter 9/50 - Loss theta2: -11.234
Iter 10/50 - Loss theta2: -11.235
Iter 11/50 - Loss theta2: -11.236
Iter 12/50 - Loss theta

In [13]:
#x0 = Tensor(np.array([0.1937, 0.1257]))
#x0 = Tensor(np.array([0.1885, 0.1038]))

x0 = Tensor(np.array([0.1942, 0.1037]))
print(vf(x0))
x0 = x0.reshape(1,2)
print(x0)
model.eval()
likelihood.eval()

pr = likelihood(model(x0))
print(pr.mean)

[0.49800091 0.99920975]
tensor([[0.1942, 0.1037]])
tensor([[0.4992, 1.0013]], grad_fn=<ViewBackward>)


In [ ]:
yy2 = np.zeros([9,2])
yy1 = vec_x.detach().numpy() 
yy2[0] = np.array([0.5, 0.7]) 
yy2[1:9] = yy1 
print(yy2) 
yy = yy2

In [ ]:
y = torch.zeros(yy.shape)
for i in range(y.shape[0]):
    y[i]=Tensor(vf(yy[i]))
print(y)

In [ ]:
y = torch.zeros(yy.shape)
for i in range(y.shape[0]):
    x0 = Tensor(yy[i])
    x0 = x0.reshape(1,2)
    model.eval()
    likelihood.eval()

    pr = likelihood(model(x0))
    y[i]=pr.mean
print(y)

In [ ]:
likelihood.noise.item()

In [ ]:
g_theta2_detach

In [ ]:
g_theta1.shape

In [ ]:
y1 = g_theta1.detach()
y2 = Tensor(loc_sample)
tol = 0.05
for ii in range (y1.shape[0]):
#         if (x[ii,0] < 0 or x[ii,1] <0):
#             check = True
#             index_del.append(ii)
    for jj in range(y2.shape[0]):
        if (torch.norm(y1[ii] - y2[jj])) <= tol:
            print(ii)
            print(jj)
            print(torch.norm(y1[ii] - y2[jj])) 

In [14]:
g_theta2

Parameter containing:
tensor([[1.0785, 0.5677],
        [0.1584, 0.0405],
        [0.2303, 1.0559],
        [0.1669, 0.0423],
        [0.1593, 0.0126],
        [1.0323, 0.6591],
        [0.1645, 0.0383],
        [0.1562, 0.0261],
        [0.4275, 0.0131],
        [0.1538, 0.0297],
        [1.0265, 0.6513],
        [0.1505, 0.1298],
        [0.1610, 0.0116],
        [0.1690, 0.1536],
        [0.1353, 0.3821],
        [1.0508, 0.6613],
        [0.1241, 0.1100],
        [0.1387, 0.3077],
        [0.1372, 0.2195],
        [1.0133, 0.7117],
        [0.1294, 0.0538],
        [1.0491, 0.6499],
        [0.1596, 0.0238],
        [1.0909, 0.5417],
        [0.2532, 1.0750],
        [0.1571, 0.0976],
        [0.1242, 0.1000],
        [0.1338, 0.3413],
        [1.0045, 0.7133],
        [0.1554, 0.0191],
        [1.0390, 0.6688],
        [0.3465, 0.0195],
        [0.9737, 0.7176],
        [0.2653, 0.0330],
        [0.3101, 0.0297],
        [1.0407, 0.6562],
        [1.0125, 0.7040],
        [0.1803,

In [ ]:
iter

In [ ]:
iter

In [ ]:
x0_new  = Tensor([[0.2771, 0.1796]])

In [ ]:
check_dist(g_theta1, x0_new, 0.0001)